# Project: Investigate Gun Data in US

## Introduction

Following the implementation of the Brady act in 1994, the Federal Bureau of Investigation (FBI) developed a system to conduct background checks on individuals wanting to obtain a firearm. The system known as the National Instant Criminal Background Check System (NICS) was created in collaboration with the Buereu of Alcohol, Tabacco and Firearms and local law enforcement agencies. Since it's inception in November 1998, the FBI has released monthly data from each state and U.S territory. The FBI claims that over 300 million requests have been aprroved, and 1.5 million have been denied.

### Questions to Investigate:
* How does `permits` changed across years?
* In the gun shooting data, to what extent do the killed vs injured people change? 
* What is the overall trend of gun purchases?
* Which states have the highest growth in gun registrations?

In [2]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import helpers as hp
import sweetviz as sv

## Data Wrangling
* I have surfed the internet for associated data that can help dig deeper in the provided data to extract insightful analysis. The data found was `Gun violence database` that is found here: https://www.kaggle.com/gunviolencearchive/gun-violence-database?select=mass_shootings_all.csv

In [4]:
# Reading the dataset
df_gun = pd.read_csv("gun_data.csv")
df_violence = pd.read_csv("mass_shootings_all.csv")

In [5]:
# Info
df_gun.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12485 entries, 0 to 12484
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   month                      12485 non-null  object 
 1   state                      12485 non-null  object 
 2   permit                     12461 non-null  float64
 3   permit_recheck             1100 non-null   float64
 4   handgun                    12465 non-null  float64
 5   long_gun                   12466 non-null  float64
 6   other                      5500 non-null   float64
 7   multiple                   12485 non-null  int64  
 8   admin                      12462 non-null  float64
 9   prepawn_handgun            10542 non-null  float64
 10  prepawn_long_gun           10540 non-null  float64
 11  prepawn_other              5115 non-null   float64
 12  redemption_handgun         10545 non-null  float64
 13  redemption_long_gun        10544 non-null  flo

In [6]:
df_violence.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Incident Date   500 non-null    object 
 1   State           500 non-null    object 
 2   City Or County  500 non-null    object 
 3   Address         498 non-null    object 
 4   # Killed        500 non-null    int64  
 5   # Injured       500 non-null    int64  
 6   Operations      0 non-null      float64
dtypes: float64(1), int64(2), object(4)
memory usage: 27.5+ KB


## Data Cleaning
* Cleaning columns names only for `df_violence`
* Data types Conversion: `df_gun['month', 'multiple', 'totals']` and `df_violence['Incident Date']` to their correct dtypes.
* Dealing with missing values